<a href="https://colab.research.google.com/github/zhouchun0105/Bitcoin-Price-and-Movement-Prediction/blob/main/bitcoin_predict_LSTM_JinhanM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive 
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
df = pd.read_csv('drive/MyDrive/bitcoin.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'],unit='s')
df = df.set_index('Timestamp')
df.columns = ["Open", "High", "Low","Close", "Volume_BTC", "Volume_Currency", "Weighted_Price"]
df.dropna(inplace=True)
df.head(10)

,Open,High,Low,Close,Volume_BTC,Volume_Currency,Weighted_Price
Timestamp,,,,,,,
2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
2011-12-31 15:50:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2011-12-31 16:59:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
2011-12-31 17:00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
2012-01-01 04:16:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
2012-01-01 15:28:00,4.84,4.84,4.84,4.84,10.000000,48.400000,4.840000
2012-01-01 22:45:00,5.00,5.00,5.00,5.00,10.100000,50.500000,5.000000
2012-01-02 20:04:00,5.00,5.00,5.00,5.00,19.048000,95.240000,5.000000
2012-01-03 11:45:00,5.32,5.32,5.32,5.32,2.419173,12.870000,5.320000


In [ ]:
df["Price"] = df[["Open","High","Low","Close"]].mean(axis=1)
df["Return"] = df["Price"].diff()
df["Percentage_Return"] = ((df["Return"].shift(periods=-1))/ (df["Price"])).shift(periods=1)
df["Volume_diff"] = df["Volume_Currency"].diff()
df["Percentage_Volume"] = ((df["Volume_diff"].shift(periods=-1))/ (df["Price"])).shift(periods=1)

In [ ]:
low, high = 0.01, 0.99
selected_columns = ["Percentage_Return", "Percentage_Volume"]

quant_df = df[selected_columns].quantile([low, high])
quant_df

,Percentage_Return,Percentage_Volume
0.01,-0.004313,-83.656998
0.99,0.004219,85.030174


In [ ]:
for col in selected_columns:
    df.drop(df[df[col] <= quant_df.loc[low, col]].index, axis=0, inplace=True)
    df.drop(df[df[col] >= quant_df.loc[high, col]].index, axis=0, inplace=True)

In [ ]:
stamp1 = pd.Timestamp(2014,6,1)
df = df[df.index>stamp1]

In [ ]:
df_t = df[["Volume_Currency", "Weighted_Price"]].copy(deep=True)
times = [2, 120, 240, 600, 1440, 14400, 28800]
for col in df_t.columns:
    for t in times:
        df_t[col+'_ratio_'+str(t)] = df_t[col]/df_t[col].rolling(window=t).mean()
df_t.dropna(inplace=True)
columns = df_t.columns.tolist()
columns = [columns[11]] + columns[:11] + columns[12:]
df_t = df_t[columns]
df_t

,Weighted_Price_ratio_240,Volume_Currency,Weighted_Price,Volume_Currency_ratio_2,Volume_Currency_ratio_120,Volume_Currency_ratio_240,Volume_Currency_ratio_600,Volume_Currency_ratio_1440,Volume_Currency_ratio_14400,Volume_Currency_ratio_28800,Weighted_Price_ratio_2,Weighted_Price_ratio_120,Weighted_Price_ratio_600,Weighted_Price_ratio_1440,Weighted_Price_ratio_14400,Weighted_Price_ratio_28800
Timestamp,,,,,,,,,,,,,,,,
2014-07-01 13:20:00,1.008167,149.643129,652.755990,0.494116,0.040897,0.049898,0.040948,0.035313,0.050734,0.038701,1.000955,1.005684,1.010230,1.025384,1.097148,1.061968
2014-07-01 13:21:00,1.008528,2037.418383,653.018712,1.863156,0.555999,0.677806,0.557247,0.480772,0.690723,0.526918,1.000201,1.005959,1.010613,1.025769,1.097577,1.062393
2014-07-01 13:23:00,1.009636,1677.587661,653.764882,0.903141,0.456137,0.556909,0.458482,0.396185,0.568723,0.433983,1.000571,1.006984,1.011743,1.026914,1.098819,1.063606
2014-07-01 13:24:00,1.006434,13034.225000,651.711250,1.771940,3.473215,4.262214,3.546248,3.071833,4.417423,3.371495,0.998427,1.003695,1.008546,1.023664,1.095355,1.060263
2014-07-01 13:25:00,1.006079,117.270000,651.500000,0.017834,0.032981,0.038602,0.031936,0.027641,0.039744,0.030334,0.999838,1.003231,1.008200,1.023307,1.094988,1.059918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,0.999584,81259.372187,58692.753339,0.708872,1.198122,1.001461,0.738670,0.588387,0.463320,0.421717,0.999666,0.999355,0.998660,1.006009,1.056144,1.036530
2021-03-30 23:57:00,0.999611,428158.146640,58693.226508,1.680971,6.051692,5.232474,3.871621,3.096415,2.440925,2.221882,1.000004,0.999378,0.998673,1.006005,1.056152,1.036537
2021-03-30 23:58:00,0.999682,100117.070370,58696.198496,0.379034,1.398912,1.226897,0.908864,0.723793,0.570747,0.519555,1.000025,0.999441,0.998728,1.006043,1.056206,1.036588


In [ ]:
time_stamp_2021 = pd.Timestamp(2021, 1, 1)
train_df = df[df.index < time_stamp_2021]
test_df = df[df.index > time_stamp_2021]
len(train_df)/len(df_t)

0.9674151438468123

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_df = scaler.fit_transform(train_df)
train_df.shape

(2902377, 12)

In [ ]:
train_df

array([[0.01583404, 0.01582218, 0.01588679, ..., 0.59519076, 0.50732711,
        0.49660061],
       [0.01583404, 0.01582218, 0.01579058, ..., 0.24414874, 0.51450412,
        0.81381142],
       [0.01573106, 0.01571921, 0.01578371, ..., 0.20614355, 0.50010984,
        0.1768978 ],
       ...,
       [0.99106147, 0.99137168, 0.9916245 , ..., 0.47117533, 0.58049669,
        0.56584927],
       [0.99124202, 0.99137134, 0.99082935, ..., 0.44238831, 0.54358537,
        0.53059991],
       [0.98910187, 0.99054651, 0.98978851, ..., 0.38032734, 0.41758953,
        0.41016096]])

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps=30):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

train_X, train_y = split_sequences(train_df)
n_features = train_X.shape[2]

In [ ]:
pip install -q pyyaml h5py

In [ ]:
checkpoint_path = 'drive/MyDrive/checkpoint/'
import tensorflow as tf
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Oct  6 18:27:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(30, n_features)))
model.add(Dropout(0.35))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(1))
# model.add(Dense(output_dim=1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
history = model.fit(train_X, train_y, batch_size=32, epochs=100, verbose=2, validation_split=0.25, callbacks=[cp_callback])

Epoch 1/100


ValueError: ignored

In [ ]:
# from numpy import array
# from numpy import hstack
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
 
# # split a multivariate sequence into samples
# def split_sequences(sequences, n_steps):
# 	X, y = list(), list()
# 	for i in range(len(sequences)):
# 		# find the end of this pattern
# 		end_ix = i + n_steps
# 		# check if we are beyond the dataset
# 		if end_ix > len(sequences):
# 			break
# 		# gather input and output parts of the pattern
# 		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
# 		X.append(seq_x)
# 		y.append(seq_y)
# 	return array(X), array(y)
 
# # define input sequence
# in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
# in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
# out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# # convert to [rows, columns] structure
# in_seq1 = in_seq1.reshape((len(in_seq1), 1))
# in_seq2 = in_seq2.reshape((len(in_seq2), 1))
# out_seq = out_seq.reshape((len(out_seq), 1))
# # horizontally stack columns
# dataset = hstack((in_seq1, in_seq2, out_seq))
# # choose a number of time steps
# n_steps = 3
# # convert into input/output
# X, y = split_sequences(dataset, n_steps)
# # the dataset knows the number of features, e.g. 2
# n_features = X.shape[2]
# # define model
# model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')
# # fit model
# model.fit(X, y, epochs=200, verbose=2)
# # demonstrate prediction
# x_input = array([[80, 85], [90, 95], [100, 105]])
# x_input = x_input.reshape((1, n_steps, n_features))
# yhat = model.predict(x_input, verbose=0)
# print(yhat)

Epoch 1/200
1/1 - 1s - loss: 16006.6982
Epoch 2/200
1/1 - 0s - loss: 15802.9131
Epoch 3/200
1/1 - 0s - loss: 15632.6973
Epoch 4/200
1/1 - 0s - loss: 15488.3213
Epoch 5/200
1/1 - 0s - loss: 15363.1689
Epoch 6/200
1/1 - 0s - loss: 15251.9863
Epoch 7/200
1/1 - 0s - loss: 15150.6953
Epoch 8/200
1/1 - 0s - loss: 15056.1387
Epoch 9/200
1/1 - 0s - loss: 14965.8574
Epoch 10/200
1/1 - 0s - loss: 14877.8076
Epoch 11/200
1/1 - 0s - loss: 14790.2715
Epoch 12/200
1/1 - 0s - loss: 14701.7676
Epoch 13/200
1/1 - 0s - loss: 14610.9639
Epoch 14/200
1/1 - 0s - loss: 14516.7363
Epoch 15/200
1/1 - 0s - loss: 14418.1113
Epoch 16/200
1/1 - 0s - loss: 14314.4189
Epoch 17/200
1/1 - 0s - loss: 14205.4395
Epoch 18/200
1/1 - 0s - loss: 14091.4268
Epoch 19/200
1/1 - 0s - loss: 13973.1123
Epoch 20/200
1/1 - 0s - loss: 13851.8857
Epoch 21/200
1/1 - 0s - loss: 13729.1768
Epoch 22/200
1/1 - 0s - loss: 13606.3574
Epoch 23/200
1/1 - 0s - loss: 13484.5859
Epoch 24/200
1/1 - 0s - loss: 13364.2031
Epoch 25/200
1/1 - 0s - l